In [2]:
import telnetlib

In [3]:
with open('official.txt', 'r') as f:
    wordle_words = [x[0] for x in [line.split(',') for line in f.read().splitlines()]]
wordle_words

['rebut',
 'sissy',
 'humph',
 'awake',
 'blush',
 'focal',
 'evade',
 'naval',
 'serve',
 'heath',
 'dwarf',
 'model',
 'karma',
 'stink',
 'grade',
 'quiet',
 'bench',
 'abate',
 'feign',
 'major',
 'death',
 'fresh',
 'crust',
 'stool',
 'colon',
 'abase',
 'marry',
 'react',
 'batty',
 'pride',
 'floss',
 'helix',
 'croak',
 'staff',
 'paper',
 'unfed',
 'whelp',
 'trawl',
 'outdo',
 'adobe',
 'crazy',
 'sower',
 'repay',
 'digit',
 'crate',
 'cluck',
 'spike',
 'mimic',
 'pound',
 'maxim',
 'linen',
 'unmet',
 'flesh',
 'booby',
 'forth',
 'first',
 'stand',
 'belly',
 'ivory',
 'seedy',
 'print',
 'yearn',
 'drain',
 'bribe',
 'stout',
 'panel',
 'crass',
 'flume',
 'offal',
 'agree',
 'error',
 'swirl',
 'argue',
 'bleed',
 'delta',
 'flick',
 'totem',
 'wooer',
 'front',
 'shrub',
 'parry',
 'biome',
 'lapel',
 'start',
 'greet',
 'goner',
 'golem',
 'lusty',
 'loopy',
 'round',
 'audit',
 'lying',
 'gamma',
 'labor',
 'islet',
 'civic',
 'forge',
 'corny',
 'moult',
 'basic',


Control Flow:

User 1 Joins, sends greeting, gets no response -> Goes into await mode

User 2 Joins, sends greeting -> User 1 reads response

User 1 picks random word -> Broadcasts word to all users for synchronization

User 1 makes guess
- if incorrect:  
  - sends modified output response ( CRATE -> [(C,0),(R,0),(A,1),(T,1),(E,2)] )  
- if correct:  
  - sends win signal (WINNER WINNER CHICKEN DINNER)

User 2 reads response:
- if win signal:
  - pick random word -> broadcast word to all users for synchronization
  - make guess
- if guess:
  - parse guess, store locally
  - check if correct, incorrect, send signal, repeat




Input Username  
arnold  
Welcome, arnold!  

--> 000 bec entered game <-- #arnold sees this 

--> 001 HAPPY <-- #new word by p1, seen by bec

--> 002 CRATE <-- #guess by p1 

--> 002 FLAGS <-- #guess by p2

--> 003 HAPPY <-- #guess by p1

--> 001 BLACK <-- #new word by loser


000 -> 002

001 -> 002

002 -> 003

003 -> 001

start -> 000 -> 003 -> 003* -> (002 -> 000 -> 003) OR -> (003)*

start -> 001 -> 003 -> read -> 003* -> 

if i guess correctly, i should go into mode 001 to get a new word. if i guess incorrectly, i should wait for the opponent's guess

In [29]:
def telnet_start():
    # Join telnet
    tn = telnetlib.Telnet(host="127.0.0.1", port=4242)

    # Greeting
    tn.write('User2 '.encode("ascii")+b"\n")
    # whenever you do a tn.write, the string needs an extra character for some reason (it gets cut off)
    
    read_val = telnet_wait(tn)

    # starting after --> in read_val and then read the next 3 digits

    flag_code = read_val[1:4]
    word = read_val[5:10]
#     print(flag_code)
    if flag_code == "000":

        telnet_000(tn)
    
    if flag_code == "001":

        telnet_001(tn, word)
        
def telnet_wait(tn):
    tn.read_until(b"-->") # Throw away unnecessary info

    return tn.read_until(b"<--").decode()
    
def telnet_000(tn):    
    #new opponent joined, start game
    out_word = "001 " + wordle_words[0].upper() + " "
    word = wordle_words[0].upper()
#     print(word)
    tn.write(out_word.encode("ascii")+b"\n")
    # goto guess routine
    telnet_003(tn, word, read=False)

def telnet_001(tn, word):
#     tn.read_until(b"-->") # Throw away unnecessary info

#     read_val = tn.read_until(b"<--").decode()
#     word = read_val[5:10]
#     print(word)
    telnet_003(tn, word)

def telnet_002(tn):
    telnet_000(tn)

def telnet_003(tn, word, read=True):
#     print('start 003')
    if read:
        tn.read_until(b"-->") # Throw away unnecessary info

        read_val = tn.read_until(b"<--").decode()
        flag_code = read_val[1:4]
        if flag_code == "002":
            telnet_002(tn)
        #opponent guessed incorrectly or you set word
        opponent_guess_word = read_val[5:10]
        print("Opponent guessed: " + opponent_guess_word)
    # goto guess routine
    output_word = ""
    while len(output_word) != 5:
        output_word = input().upper()
    if output_word == word:
        tn.write('002 ***** '.encode("ascii")+b"\n")
        tn.read_until(b"-->") # Throw away unnecessary info

        read_val = tn.read_until(b"<--").decode()
        word = read_val[5:10]
        telnet_001(tn, word)
    else:
        tn.write(("003 "+output_word + " ").encode("ascii")+b"\n")
        telnet_003(tn, word)


In [ ]:
telnet_start()

Opponent guessed: AAAAA
aaaaa
REBUT
rebut
